In [1]:
import os
import pickle

import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf # Version 1.9

os.environ["CUDA_VISIBLE_DEVICES"]="0" # For training on a GPU.

%matplotlib inline


class MacOSFile():

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        print("Reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                print("Reading bytes [%s,%s)..." % (idx, idx + batch_size),
                      end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                print("Done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size),
                  end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_load(file_path):
    """Wrapper of pickle.load"""
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [2]:
# Load train and test sets.
tuning_hyperparams = False
train_X = pickle_load("../data/pickles/train_X.npy")
train_y = pickle_load("../data/pickles/train_y.npy")
if tuning_hyperparams:
    test_X = pickle_load("../data/pickles/validation_X.npy")
    test_y = pickle_load("../data/pickles/validation_y.npy")
else:
    test_X = pickle_load("../data/pickles/test_X.npy")
    test_y = pickle_load("../data/pickles/test_y.npy")

Reading total_bytes=11
Reading total_bytes=8213102745
Reading bytes [0,1073741824)...Done.
Reading bytes [1073741824,2147483648)...Done.
Reading bytes [2147483648,3221225472)...Done.
Reading bytes [3221225472,4294967296)...Done.
Reading bytes [4294967296,5368709120)...Done.
Reading bytes [5368709120,6442450944)...Done.
Reading bytes [6442450944,7516192768)...Done.
Reading bytes [7516192768,8213102745)...Done.
Reading total_bytes=334347
Reading total_bytes=11
Reading total_bytes=2566914197
Reading bytes [0,1073741824)...Done.
Reading bytes [1073741824,2147483648)...Done.
Reading bytes [2147483648,2566914197)...Done.
Reading total_bytes=104603


In [3]:
# Get shape of training set.
print("Training set (images) shape: {shape}".format(shape=train_X.shape))
print("Training set (labels) shape: {shape}".format(shape=train_y.shape))
# Get shape of test set.
print("Test set (images) shape: {shape}".format(shape=test_X.shape))
print("Test set (labels) shape: {shape}".format(shape=test_y.shape))
# Set input details.
image_size = 128
num_channels = 3
num_classes = 2
X = tf.placeholder("float", [None, image_size, image_size, num_channels])
y = tf.placeholder("float", [None, num_classes])

Training set (images) shape: (20887, 128, 128, 3)
Training set (labels) shape: (20887, 2)
Test set (images) shape: (6528, 128, 128, 3)
Test set (labels) shape: (6528, 2)


In [4]:
# Set key hyperparameters.
training_iters = 200
learning_rate = 0.001
batch_size = 128

In [14]:
def conv2d(X, W, b, stride=1):
    """tf.nn.conv2d() wrapper, with bias and relu activation."""
    # The first and last element of "strides" is example and
    # channel stride respectively.
    X = tf.nn.conv2d(X, W,
                     strides=[1, stride, stride, 1],
                     padding="SAME")
    X = tf.nn.bias_add(X, b)
    return tf.nn.relu(X)

def max_pool(X, kernel_size=2):
    # Stride of the kernel is always >= its size to prevent
    # overlap of pooling region.
    return tf.nn.max_pool(X,
                          ksize=[1, kernel_size, kernel_size, 1],
                          strides=[1, kernel_size, kernel_size, 1],
                          padding="SAME")

In [15]:
# Create dictionaries for the weight and bias parameters.
weights = {
    "wc1": tf.get_variable("W0",
                           shape=(3,3,1,32),
                           initializer=tf.contrib.layers.xavier_initializer()), 
    "wc2": tf.get_variable("W1",
                           shape=(3,3,32,64),
                           initializer=tf.contrib.layers.xavier_initializer()), 
    "wc3": tf.get_variable("W2",
                           shape=(3,3,64,128),
                           initializer=tf.contrib.layers.xavier_initializer()), 
    "wd1": tf.get_variable("W3",
                           shape=(4*4*128,128),
                           initializer=tf.contrib.layers.xavier_initializer()), 
    "out": tf.get_variable("W6",
                           shape=(128,n_classes),
                           initializer=tf.contrib.layers.xavier_initializer()), 
}

biases = {
    "bc1": tf.get_variable("B0",
                           shape=(32),
                           initializer=tf.contrib.layers.xavier_initializer()),
    "bc2": tf.get_variable("B1",
                           shape=(64),
                           initializer=tf.contrib.layers.xavier_initializer()),
    "bc3": tf.get_variable("B2",
                           shape=(128),
                           initializer=tf.contrib.layers.xavier_initializer()),
    "bd1": tf.get_variable("B3",
                           shape=(128),
                           initializer=tf.contrib.layers.xavier_initializer()),
    "out": tf.get_variable("B4",
                           shape=(10),
                           initializer=tf.contrib.layers.xavier_initializer()),
}

In [16]:
def conv_net(X, weights, biases):  
    # here we call the conv2d function we had defined above and pass the input image X, weights wc1 and bias bc1.
    conv1 = conv2d(X, weights['wc1'], biases['bc1'])
    # MaX Pooling (down-sampling), this chooses the maX value from a 2*2 matrix window and outputs a 14*14 matriX.
    conv1 = max_pool(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image X, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matriX.
    conv2 = max_pool(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = max_pool(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [18]:
pred = conv_net(X, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [19]:
#Here you check whether the index of the maximum value of the predicted image is equal
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
# Initializing the variables
init = tf.global_variables_initializer()

In [23]:
with tf.Session() as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(len(train_X)//batch_size):
            batch_X = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))] 
            # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={X: batch_X, y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={X: batch_X, y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc))
        print("Optimization Finished!")
        
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={X: test_X,y : test_y})
        train_loss.append(loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.327325, Training Accuracy= 0.87500
Optimization Finished!
Testing Accuracy: 0.84800



KeyboardInterrupt



In [ ]:
plt.plot(range(len(train_loss)), train_loss, 'b', label='Training loss')
plt.plot(range(len(train_loss)), test_loss, 'r', label='Test loss')
plt.title('Training and Test loss')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.legend()
plt.figure()
plt.show()

In [ ]:
plt.plot(range(len(train_loss)), train_accuracy, 'b', label='Training Accuracy'
plt.plot(range(len(train_loss)), test_accuracy, 'r', label='Test Accuracy')
plt.title('Training and Test Accuracy')
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.legend()
plt.figure()